<a href="https://colab.research.google.com/github/BenRoche18/Im2Calories/blob/master/food_classification_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

import os
import matplotlib.pyplot as plt
import numpy as np
import copy
import shutil

**Mount google drive**

Mount google drive that contains project files and set to working directory.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = os.path.join("drive", "My Drive", "Im2Calories")
os.chdir(PROJECT_DIR)

**Enable GPU**

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Running on", device)

**Declare parameters**

In [0]:
IMAGE_SIZE = 299
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001

# Input data

**Download Food101 dataset**

I will be using the labelled food-101 dataset that includes 101 different classes each containing 1000 images.

In [0]:
FOOD101_DIR = os.path.join("food-101")

if "food-101" not in os.listdir():
  print("Downloading Food-101 dataset... ", end="")
  !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
  print("Done")
  print("Extracting data... ", end="")
  !tar xzvf food-101.tar.gz
  print("Done")

**Split dataset into training and validation images**

The dataset comes with a train and testing split defined in two seperate text files, however for faster access we will copy the respective images into train and test directories.

In [0]:
FOOD101_DIR = "food-101"
TRAIN_DIR = os.path.join(FOOD101_DIR, "train")
VAL_DIR = os.path.join(FOOD101_DIR, "test")

print("Spliting dataset... ", end="")
images = {}

for split in ["train", "test"]:
  # extract image filenames
  with open(os.path.join(FOOD101_DIR, "meta", split+".txt"), 'r') as file:
    for path in file.readlines():
      path = path.strip()
      food_class, food_id = path.split('/')
      images.setdefault(food_class, []).append(food_id)

  # copy images into split directory
  for food_class in images.keys():
    # make class directory
    class_dir = os.path.join(FOOD101_DIR, split, food_class)
    if not os.path.exists(class_dir):
      os.makedirs(class_dir)

    # populate class directory with images
    for food_id in images[food_class]:
      src = os.path.join(FOOD101_DIR, "images", food_class, food_id+".jpg")
      dest = os.path.join(FOOD101_DIR, split, food_class, food_id+".jpg")
      if not os.path.exists(dest):
        shutil.copy(src, dest)

  images = {}

print("Done")

**Format training images**

In [0]:
train_transform = transforms.Compose([
  transforms.RandomResizedCrop(IMAGE_SIZE),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_raw = torchvision.datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
train_size = len(train_raw)
print(train_size)

train_loader = torch.utils.data.DataLoader(train_raw, batch_size=BATCH_SIZE, shuffle=True)

class_names = train_raw.classes

**Format validation images**

In [0]:
val_transform = transforms.Compose([
  transforms.Resize(350),
  transforms.CenterCrop(IMAGE_SIZE),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_raw = torchvision.datasets.ImageFolder(VAL_DIR, transform=val_transform)
val_size = len(val_raw)

val_loader = torch.utils.data.DataLoader(val_raw, batch_size=BATCH_SIZE)

**Show example images from dataset**

I have randomly selected and displayed 20 images from the training dataset along with their corresponding class.

In [0]:
cols, rows = 5, 4
fig = plt.figure(figsize=(20,10))
fig.suptitle("Random images from dataset")

for i in range(1, cols*rows+1):
  fig.add_subplot(rows, cols, i)

  # randomly sellect image from dataset
  j = np.random.randint(train_size)
  img = train_raw[j][0].numpy()
  img = np.transpose(img, (1,2,0))
  img = img * (0.485, 0.456, 0.406) + (0.229, 0.224, 0.225)

  plt.title(class_names[train_raw[j][1]])
  plt.axis("off")
  img = np.clip(img, 0, 1)
  plt.imshow(img, interpolation="nearest")
plt.show()

# Model

**Fetch Model**

In order to train the food classifier I will initially use transfer learning from a pretrained InceptionV3 model. This involves replacing the classification head with a 101-way dense layer and freezing all other layers.

In [0]:
# fetch pre-trained feature extractor
model = torchvision.models.inception_v3(pretrained=True)

# freeze feature extractor
for param in model.parameters():
  param.requires_grad = False

model.aux_logits = False

# extract number of features that are outputted in base model
num_features = model.fc.in_features

# replace classification head with 101-way dense layer
model.fc = torch.nn.Linear(num_features, 101)

model = model.to(device)

# Training via transfer learning

**Declare optimizer and loss function**

In [0]:
optimizer = torch.optim.SGD(model.fc.parameters(), lr=LEARNING_RATE, momentum=0.9)
loss_func = torch.nn.CrossEntropyLoss()

**Train model**

In [0]:
statistics = {
    "accuracy": [],
    "val_accuracy": []
}

def train():
  best_acc = 0.0
  best_model_weights = None

  print("Training on {} images...\n".format(train_size))

  for epoch in range(1, EPOCHS+1):
    print("Epoch {}/{}...".format(epoch, EPOCHS))

    model.train()
    running_loss = 0.0
    running_corrects = 0

    for (images, labels) in train_loader:
      images = images.to(device)
      labels = labels.to(device)

      # reset the parameter gradients
      optimizer.zero_grad()

      # fit images on model
      outputs = model(images)
      _, predictions = torch.max(outputs, 1)
      loss = loss_func(outputs, labels)

      loss.backward()
      optimizer.step()

      running_loss += loss.item() * images.size(0)
      running_corrects += torch.sum(predictions == labels.data)

    # calculate statistics
    epoch_loss = running_loss / train_size
    epoch_acc = running_corrects.double() / train_size
    
    # print statistics
    print("Training Loss: {:.4f}, Acc: {:.4f}".format(epoch_loss, epoch_acc))

    val_loss, val_acc = validate()

    # save best model seen
    if val_acc > best_acc:
      best_acc = epoch_acc
      best_model_weights = copy.deepcopy(model.state_dict())

    # print statistics
    print("Validation Loss: {:.4f}, Acc: {:.4f}\n".format(val_loss, val_acc))

  # reinstantiate best seen weights
  model.load_state_dict(best_model_weights)

  print("DONE")


def validate():
  model.eval()
  running_loss = 0.0
  running_corrects = 0

  for (images, labels) in val_loader:
    images = images.to(device)
    labels = labels.to(device)

    # fit images on model
    with torch.no_grad():
      outputs = model(images)
      _, predictions = torch.max(outputs, 1)
      loss = loss_func(outputs, labels)

    running_loss += loss.item() * images.size(0)
    running_corrects += torch.sum(predictions == labels.data)

  # calculate statistics
  val_loss = running_loss / val_size
  val_acc = running_corrects.double() / val_size

  return val_loss, val_acc


train()

**Save model**

In [0]:
MODEL_PATH = os.path.join("models", "food_classifier_TL_v1.pt")

torch.save(model.state_dict(), MODEL_PATH)

**Evaluate transfer learning**

In [0]:
plt.plot(statistics['accuracy'], label='accuracy')
plt.plot(statistics['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

# Training via fine-tuning

**Unfreeze layers in model**

In [0]:
for param in model.parameters():
  param.requires_grad = True

EPOCHS = 5

**Train model**

In [0]:
statistics = {
    "accuracy": [],
    "val_accuracy": []
}

train()

**Save model**

In [0]:
MODEL_PATH = os.path.join("models", "food_classifier_FT_v2")

torch.save(model.state_dict(), MODEL_PATH)

# Evaluate

In [0]:
plt.plot(statistics['accuracy'], label='accuracy')
plt.plot(statistics['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')